# Seq2Seq : Sequence to Sequence 모델
  
  
## : Encoder Decoder 모델이라고도 한다

### Encoder 클래스

In [2]:
from nn_layers import TimeEmbedding, TimeLSTM, TimeAffine, TimeSoftmaxWithLoss


In [4]:
class Encoder:
    def __init__(self,vocab_size, wordvec_size, hidden_size):
        
        V,D,H = vocab_size, wordvec_size,hidden_size         # 10000, 100, 100
        rn = np.random.randn

        # 가중치 초기화
        embed_W = (rn(V,D)/100).astype('f')

        # Wx, Wh, b가 4*H로 된다. 
        lstm_Wx = (rn(D,4*H) / np.sqrt(D)).astype('f') 
        lstm_Wh = (rn(H,4*H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4*H).astype('f')
        
        # 계층 생성
        self.embed = TimeEmbedding(embed_W)
        
        # 이전의 상태를 쓰지 않음.
        # Encoder 측에서는 처음 state가 없는 상태로 시작.
        # stateful이 false니까 h는 0로 들어옴 (TimeLSTM에서 if문으로 설정)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False) # stateful False
        
        # 모든 가중치와 기울기를 모은다.
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        
    # x가 들어오면, 출력은 h만 나간다.
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)      # (N, T, H)
        
        self.hs = hs
        
        # TimeLSTM의 마지막 계층만 반환,.
        # TimeLSTM 게층의 마지막 은닉 상태 h를 반환, [N, H], 2차원
        return hs[:, -1, :]
        